In [18]:
import os
import json
import pandas as pd
import traceback

from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from dotenv import load_dotenv

load_dotenv()
key = os.getenv("groq_api")

llm = ChatGroq(groq_api_key=key, temperature=1, model="qwen-2.5-32b")

template = """
You are an MCQ generator.
Topic: {Topic}
Text: {Text}

If the text is None, generate MCQs based on the given topic.
If the provided text is insufficient, use your knowledge to generate relevant MCQs.

Now create {Count} MCQs in {Mode} mode as per the given topic.
The response should be in JSON format:

### Response format:
{json_format}
"""

response_format = {
    "1": {
        "Question": "question",
        "options": {
            "a": "choice 1",
            "b": "choice 2",
            "c": "choice 3",
            "d": "choice 4",
        },
    },
    "2": {},
}

answers_format = {
    "1": {
        "answer": "correct option",
        "reason": "Provide a small description about why that option is correct.",
    },
    "2": {},
}

answers_template = """
MCQs: {generated_mcqs}

As per the above, provide correct answers in the following format:

## Result format:
{answers_format}
"""

def generate_mcqs(topic, text=None, count=5, mode="low"):
    quiz_generate_prompt = PromptTemplate(
        input_variables=["Topic", "Text", "Count", "Mode", "json_format"],
        template=template,
    )

    quiz_answers_prompt = PromptTemplate(
        input_variables=["generated_mcqs"],
        template=answers_template,
    )

    chain1 = LLMChain(llm=llm, prompt=quiz_generate_prompt, output_key="generated_mcqs")
    chain2 = LLMChain(llm=llm, prompt=quiz_answers_prompt, output_key="mcq_answers")

    main_chain = SequentialChain(
        chains=[chain1, chain2],
        input_variables=["Topic", "Text", "Count", "Mode", "json_format", "answers_format"],
        output_variables=["generated_mcqs", "mcq_answers"],
    )

    response = main_chain.invoke(
        {
            "Topic": topic,
            "Text": None,  # Ensure `text` is defined or replace with `None`
            "Count": count,
            "Mode": mode,
            "json_format": response_format,
            "answers_format": answers_format,
        }
    )

    raw_json = response["generated_mcqs"]

    if raw_json.startswith("```json"):
        raw_json = raw_json[7:]
    if raw_json.endswith("```"):
        raw_json = raw_json[:-3]

    try:
        mcqs = json.loads(raw_json.strip())
    except json.JSONDecodeError:
        mcqs = raw_json

    raw_answers_json = response['mcq_answers']

    if raw_answers_json.startswith("```json"):
        raw_answers_json = raw_answers_json[7:]
    if raw_answers_json.endswith("```"):
        raw_answers_json = raw_answers_json[:-3]

    
    mcqs_answers = json.loads(raw_answers_json)

    return mcqs, mcqs_answers




In [20]:
import json
import time

def safe_generate_mcqs(topic, text=None, count=5, mode="low", retries=5, delay=2):
    for attempt in range(1, retries + 1):
        try:
            return generate_mcqs(topic, text, count, mode)
        except json.JSONDecodeError as e:
            print(f"JSON error: {e}. Retrying {attempt}/{retries}...")
        except Exception as e:
            print(f"Error: {e}. Retrying {attempt}/{retries}...")
        time.sleep(delay)
    raise RuntimeError("MCQ generation failed after multiple attempts.")

# Call the function
a, b = safe_generate_mcqs("Python", count=5, mode="low")



In [21]:
print(a)


{'1': {'Question': 'What is the result of the expression 2 + 3 * 4 in Python?', 'options': {'a': '20', 'b': '14', 'c': '5', 'd': '11'}}, '2': {'Question': 'Which of the following is the correct way to comment a single line in Python?', 'options': {'a': '# This is a comment', 'b': '-- This is a comment', 'c': '// This is a comment', 'd': '/* This is a comment */'}}, '3': {'Question': 'What will be the output of the following code?\n```python\nx = 3\nprint(x ** 3)\n```', 'options': {'a': '27', 'b': '9', 'c': '3', 'd': '6'}}, '4': {'Question': 'In Python, which of the following is the correct method to create an empty list?', 'options': {'a': 'my_list = {}', 'b': 'my_list = ()', 'c': 'my_list = []', 'd': "my_list = ''"}}, '5': {'Question': "What does the 'len()' function do in Python?", 'options': {'a': 'It returns the largest item in an iterable', 'b': 'It returns the smallest item in an iterable', 'c': 'It returns the sum of all items in an iterable', 'd': 'It returns the number of item

In [22]:
print(b)

{'1': {'answer': 'b', 'reason': 'The expression `2 + 3 * 4` in Python follows the arithmetic rules with multiplication having higher precedence over addition. Therefore, it computes as 2 + (3 * 4) which is 14.'}, '2': {'answer': 'a', 'reason': 'In Python, a single line comment is started with a single hash symbol `#`. This means that `# This is a comment` is the correct format for a single line comment.'}, '3': {'answer': 'a', 'reason': 'The code `x = 3` assigns the value 3 to the variable x. Applying the operation `x ** 3` is raising x to the power of 3, which would result in 27.'}, '4': {'answer': 'c', 'reason': "To create an empty list in Python, square brackets `[]` are used. Other options are used to create dictionaries `{}`, tuples `()`, and strings `''` respectively."}, '5': {'answer': 'd', 'reason': 'The `len()` function in Python returns the number of items in an iterable. It counts the number of elements in the list, string, tuple, set, or dictionary.'}}
